In [3]:
from keras import layers, models
from keras.applications import VGG16
from keras import Input
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy import io
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer

Using TensorFlow backend.


In [8]:
def get_model_simple():
    input_tensor = Input(shape=(96, 48, 1), dtype='float32', name='input')
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(input_tensor)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D((2,2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D((2,2))(x)
    
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D((2,2))(x)
    
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D((2,2))(x)
    
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D((2,2))(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(4096, kernel_initializer='he_normal')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(4096, kernel_initializer='he_normal')(x)
    x = layers.Dropout(0.4)(x)
    output_tensor = layers.Dense(3, activation='softmax')(x)
    
    myvgg = Model(input_tensor, output_tensor)
    myvgg.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])
    myvgg.summary()
    return myvgg

In [9]:
def get_model_multi():
    input_tensor = Input(shape=(96, 48, 1), dtype='float32', name='input')
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(input_tensor)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D((2,2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D((2,2))(x)
    
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D((2,2))(x)

    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D((2,2))(x)
    
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.MaxPooling2D((2,2))(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(4096, kernel_initializer='he_normal')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(4096, kernel_initializer='he_normal')(x)
    x = layers.Dropout(0.3)(x)
    output_tensor = layers.Dense(6, activation='sigmoid')(x)
    
    myvgg = Model(input_tensor, output_tensor)
    myvgg.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])
    myvgg.summary()
    return myvgg

In [10]:
def probaToOHE(pred_test):
    for proc in pred_test:
        if proc[2] >= proc[4] and proc[2] >= proc[5]:
            proc[2] = 1
            proc[4] = 0
            proc[5] = 0
        elif proc[4] >= proc[2] and proc[4] >= proc[5]:
            proc[2] = 0
            proc[4] = 1
            proc[5] = 0
        elif proc[5] >= proc[4] and proc[5] >= proc[2]:
            proc[2] = 0
            proc[4] = 0
            proc[5] = 1
    for proc in pred_test:
        if proc[0] >= proc[1] and proc[0] >= proc[3]:
            proc[0] = 1
            proc[1] = 0
            proc[3] = 0
        elif proc[1] >= proc[0] and proc[1] >= proc[3]:
            proc[0] = 0
            proc[1] = 1
            proc[3] = 0
        elif proc[3] >= proc[1] and proc[3] >= proc[0]:
            proc[0] = 0
            proc[1] = 0
            proc[3] = 1
    return pred_test

# nor + rbbb + lbbb and sinus + pvc + vt

In [13]:
lead = 'V1'

model = get_model_multi()
file_path="my_ecg_multilable_"+lead+".h5"
model.load_weights(file_path)

Test = io.loadmat('./testImg_half_'+lead+'.mat')
X, y = Test['data'], Test['label']
X = X/255.0
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)

X_reshape = X.reshape(X.shape[0], 96, 48, 1)
pred_test = model.predict(X_reshape)

Pred = probaToOHE(pred_test)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 96, 48, 1)         0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 96, 48, 64)        640       
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 96, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 48, 24, 64)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 48, 24, 128)       73856     
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 48, 24, 128)       147584    
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 24, 12, 128)       0   

In [14]:
accuracy_score(Pred, y)

0.8757679180887372

In [16]:
f1_score(Pred, y, average="macro")

0.8968592241853292

# nor + rbbb + lbbb

In [17]:
model = get_model_simple()
file_path="my_ecg.h5"
model.load_weights(file_path)

Test = io.loadmat('./testImg_half_shape_V1.mat')
X, y = Test['data'], Test['label']
X = X/255.0

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

X_reshape = X.reshape(X.shape[0], 96, 48, 1)
pred_test = model.predict(X_reshape)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 96, 48, 1)         0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 96, 48, 64)        640       
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 96, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 48, 24, 64)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 48, 24, 128)       73856     
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 48, 24, 128)       147584    
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 24, 12, 128)       0   

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [19]:
pred_test_label = np.argmax(pred_test, axis=-1)

In [21]:
accuracy_score(pred_test_label, y)

0.9121771217712177

In [23]:
f1_score(pred_test_label, y, average="macro")

0.9065350940743063